<a href="https://colab.research.google.com/github/SabadModi/MultilingualSentimentAnalysis/blob/main/Multilingual_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Install Required Libraries

In [1]:
!pip install transformers gradio tensorflow numpy librosa vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 34.0 MB/s 
     |████████████████████████████████| 5.6 MB 58.9 MB/s 
     |████████████████████████████████| 125 kB 73.6 MB/s 
     |████████████████████████████████| 6.6 MB 59.3 MB/s 
     |████████████████████████████████| 596 kB 69.7 MB/s 
     |████████████████████████████████| 101 kB 11.0 MB/s 
     |████████████████████████████████| 84 kB 2.7 MB/s 
     |████████████████████████████████| 54 kB 1.4 MB/s 
     |████████████████████████████████| 2.3 MB 56.0 MB/s 
     |████████████████████████████████| 54 kB 3.1 MB/s 
     |████████████████████████████████| 141 kB 56.1 MB/s 
     |████████████████████████████████| 57 kB 1.6 MB/s 
     |████████████████████████████████| 84 kB 1.4 MB/s 
     |████████████████████████████████| 212 kB 70.3 MB/s 
     |████████████████████████████████| 275 kB 60.9 MB/s 
     |████████████████████████████████| 6

In [2]:
import gradio as gr
import tensorflow as tf 
import numpy as np
import librosa
import time
from transformers import pipeline
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# 2. Import the pipeline for ASR

In [3]:
p = pipeline("automatic-speech-recognition")

No model was supplied, defaulted to facebook/wav2vec2-base-960h and revision 55bb623 (https://huggingface.co/facebook/wav2vec2-base-960h).
Using a pipeline without specifying a model name and revision in production is not recommended.


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# 3. Create Detection Functions

### Create a folder called `models` and import the 3 model files into the folder

## Functions for Feature Extraction Models

In [4]:
def onlyEnglish(filename):
    model = tf.keras.models.load_model("models/English.hdf5")
    class_names = ['Anger','Anxious','Apologetic','Concerned','Encouraging','Excited','Happiness','Sadness']
    audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)
    predicted_label=model.predict(mfccs_scaled_features)
    classes_x=np.argmax(predicted_label,axis=1)
    class_num = classes_x[0]
    return class_names[class_num]

def englishGerman(filename):
    model = tf.keras.models.load_model("models/English-German.hdf5")
    class_names = ['Anger','Happiness','Sadness']
    audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)
    predicted_label=model.predict(mfccs_scaled_features)
    classes_x=np.argmax(predicted_label,axis=1)
    class_num = classes_x[0]
    return class_names[class_num]

def multiple(filename):
    model = tf.keras.models.load_model("models/Multiple.hdf5")
    class_names = ["Anger","Disgust","Fear","Happiness","Neutral","Sadness"]
    audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)
    predicted_label=model.predict(mfccs_scaled_features)
    classes_x=np.argmax(predicted_label,axis=1)
    class_num = classes_x[0]
    return class_names[class_num]

## Functions for Speech-to-Text Model

In [5]:
def sentiment_vader(sentence):

    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()

    sentiment_dict = sid_obj.polarity_scores(sentence)
    negative = sentiment_dict['neg']
    neutral = sentiment_dict['neu']
    positive = sentiment_dict['pos']
    compound = sentiment_dict['compound']

    if sentiment_dict['compound'] >= 0.05 :
        overall_sentiment = "Positive"

    elif sentiment_dict['compound'] <= - 0.05 :
        overall_sentiment = "Negative"

    else :
        overall_sentiment = "Neutral"

    return overall_sentiment

def transcribe(audio, state=""):
    time.sleep(3)
    text = p(audio)["text"]
    text = sentiment_vader(text)
    return text

# 4. Define Gradio Interface

In [6]:
with gr.Blocks() as interface:
    gr.Markdown("<h1 style='text-align: center'>Deployed 3 different Models for different types of languages.</h1><ol><li>Model 1 has 8 Classes of Emotions and is trained only on English Language. This model extracts features and trains a NN model upon that.</li><li>Model 2 has 3 Classes of Emotions and is trained only on English and German. This model extracts features and trains a NN model upon that.</li><li>Model 3 has 6 Classes of Emotions and is trained only on English, French, German and Mexican. This model extracts features and trains a NN model upon that.</li><li>Model 4 has 3 Classes of Emotions and is trained only on English. It converts Speech-to-Text and then performs Sentiment Analysis.</li></ol>")
    with gr.Tabs():
        with gr.TabItem("Only English Model"):
            english_input = gr.inputs.Audio(label="Input Audio", type="filepath")
            english_button = gr.Button("Submit")
            english_output = gr.outputs.Label(num_top_classes = 8)
            
        with gr.TabItem("English-German Model"):
            two_input = gr.inputs.Audio(label="Input Audio", type="filepath")
            two_button = gr.Button("Submit")
            two_output = gr.outputs.Label(num_top_classes = 3)
            
        with gr.TabItem("English-French-German-Mexican Model"):
            multiple_input = gr.inputs.Audio(label="Input Audio", type="filepath")
            multiple_button = gr.Button("Submit")
            multiple_output = gr.outputs.Label(num_top_classes = 6)
        
        with gr.TabItem("Speech to Text"):
            stt_input = gr.inputs.Audio(label="Input Audio", type="filepath")
            stt_button = gr.Button("Submit")
            stt_output = gr.outputs.Label(num_top_classes = 6)
      
    english_button.click(onlyEnglish, inputs=english_input, outputs=english_output)
    two_button.click(englishGerman, inputs=two_input, outputs=two_output)
    multiple_button.click(englishGerman, inputs=multiple_input, outputs=multiple_output)
    stt_button.click(transcribe, inputs=stt_input, outputs=stt_output)

/usr/local/lib/python3.7/dist-packages/gradio/inputs.py:319: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  "Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your components from gradio.components",
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.7/dist-packages/gradio/outputs.py:197: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  "Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components",
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.

# 5. Launch the Interface

In [7]:
interface.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://59098.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x7f12d7992fd0>,
 'http://127.0.0.1:7860/',
 'https://59098.gradio.app')